In [1]:
%pylab inline
import h5py
import mentari_v2 as mtr
import os

Populating the interactive namespace from numpy and matplotlib


In [ ]:
directory_dusty = "../dusty-sage/src/auxdata/trees/save-stellar-DTG/"
snap_limit = 63
BoxSize = ((62.5**3) * (8/8))**(1/3) #mini-millennium
firstfile = 0
lastfile = 7
Hubble_h = 0.73
#Generate SED
Age = np.asarray([0.0124, 0.0246, 0.0491, 0.1037, 0.1871, 0.2120, 0.2399, 0.2709, 0.3054, 0.3438, 0.3864, 0.4335, 0.4856, 0.5430, 0.6062, 0.6756, 0.7517, 0.8349, 0.9259, 1.0249, 1.1327, 1.2496, 1.3763, 1.5131, 1.6606, 1.8192, 1.9895, 2.1717, 2.3662, 2.5734, 2.7934, 3.0265, 3.2726, 3.5318, 3.8038, 4.0886, 4.3856, 4.6944, 5.0144, 5.3488, 5.6849, 6.0337, 6.3901, 6.7531, 7.1215, 7.4940, 7.8694, 8.2464, 8.6238, 9.0004, 9.3750, 9.7463, 10.1133, 10.4750, 10.8303, 11.1783, 11.5181, 11.8490, 12.1702, 12.4811, 12.7810, 13.0695, 13.3459, 13.6098])


In [ ]:
mass_dusty, metals_dusty = mtr.build_mass_and_metallicity_history(1, directory_dusty, firstfile, lastfile, snap_limit)
dust, gas_metals, gas, rad  = mtr.build_dust_history(1, directory_dusty, firstfile, lastfile, snap_limit)

In [ ]:
#Compute attenuation parameters
#w = np.where((mass_dusty[:,snap_limit] > 0) & (dust[:,snap_limit] > 0))[0]
w = np.where(mass_dusty[:,snap_limit] > 0)[0]
Mass = mass_dusty[w] / Hubble_h 
Metals = metals_dusty[w]

Dust = dust[w,snap_limit] / Hubble_h
Gas = gas[w,snap_limit] / Hubble_h
Rad = rad[w,snap_limit] / Hubble_h
eta_BC = [-0.7] * len(Dust)
eta_ISM_v2 = [-1.3] * len(Dust)

prescription = 0 #0 for Lagos+ 19; 1 for Somerville+ 12
tau_BC, eta_BC, tau_ISM, eta_ISM = mtr.compute_attenuation_parameters (prescription, Dust, Gas, Rad)


In [ ]:
Age = np.asarray([0.0124, 0.0246, 0.0491, 0.1037, 0.1871, 0.2120, 0.2399, 0.2709, 0.3054, 0.3438, 0.3864, 0.4335, 0.4856, 0.5430, 0.6062, 0.6756, 0.7517, 0.8349, 0.9259, 1.0249, 1.1327, 1.2496, 1.3763, 1.5131, 1.6606, 1.8192, 1.9895, 2.1717, 2.3662, 2.5734, 2.7934, 3.0265, 3.2726, 3.5318, 3.8038, 4.0886, 4.3856, 4.6944, 5.0144, 5.3488, 5.6849, 6.0337, 6.3901, 6.7531, 7.1215, 7.4940, 7.8694, 8.2464, 8.6238, 9.0004, 9.3750, 9.7463, 10.1133, 10.4750, 10.8303, 11.1783, 11.5181, 11.8490, 12.1702, 12.4811, 12.7810, 13.0695, 13.3459, 13.6098])
time_BC = 10**7
SSP = 0 #0 for BC03
wavelength, spectra, spectra_dusty = mtr.generate_SED(0, Age, Mass, Metals, 
             tau_BC, tau_ISM, eta_BC, eta_ISM, time_BC)

In [ ]:
wavelength_IR, spectra_IR = mtr.add_IR_Dale(wavelength, spectra, spectra_dusty)

In [ ]:
wave_FIR, spec_FIR = mtr.compute_IR_SUNRISE (Dust, wavelength, spectra, spectra_dusty)

In [ ]:
len(spec_FIR), len(spectra_IR)

In [ ]:
wavelength_IR_b, spectra_IR_b = mtr.combine_Dale_SUNRISE(Dust, wavelength, spectra, spectra_dusty)

In [ ]:
output = "output/mentari_output_" + str(1) + ".hdf5"
with h5py.File(output, 'w') as f:
    if os.path.isfile(output) == 0
    f.create_dataset('StellarMass', data=mass_dusty, maxshape=(None,snap_limit+1), chunks=True, compression="gzip")


In [ ]:
import os.path
output = "output/mentari_output_" + str(0) + ".hdf5"
test = os.path.isfile(output) 
if os.path.isfile(output):
    print ("ok")

In [ ]:
with h5py.File('mentari_test7.h5', 'a') as f:
    f.create_dataset('StellarMass', data=Mass, maxshape=(None,snap_limit+1))
    f.create_dataset('Metallicity', data=Metals, maxshape=(None,snap_limit+1))
    f.create_dataset('DustMass', data=Dust, maxshape=(None,))
    f.create_dataset('GasMass', data=Gas, maxshape=(None,))
    f.create_dataset('Radius', data=Rad, maxshape=(None,))
    f.create_dataset('Wavelength_UVIR', data=wavelength_IR, maxshape=(None,))
    f.create_dataset('Spectra_UVIR', data=spectra_IR, maxshape=(None,len(wavelength_IR)))
    f.create_dataset('Wavelength_stellar', data=wavelength, maxshape=(None,))
    f.create_dataset('Spectra_stellar', data=spectra, maxshape=(None,len(wavelength)))
    f.create_dataset('Wavelength_SUNRISE', data=wavelength_IR_b, maxshape=(None,))
    f.create_dataset('Spectra_SUNRISE', data=spectra_IR_b, maxshape=(None,len(wavelength_IR_b)))


## Process per file -- and save per file as well

## reading mentari output

In [8]:
directory_dusty = "../dusty-sage/src/auxdata/trees/save-stellar-DTG/"
snap_limit = 63
firstfile = 0
lastfile = 0
Hubble_h = 0.73
directory_output = "output/"

save_spectra(directory_dusty, firstfile, lastfile, snap_limit, directory_output, Hubble_h)

Running file number  0
0 %.....................10 %.....................20 %.....................30 %.....................40 %.....................50 %.....................60 %.....................70 %.....................80 %.....................90 %.....................0 %.....................10 %.....................20 %.....................30 %.....................40 %.....................50 %.....................60 %.....................70 %.....................80 %.....................90 %.....................0 %.....................10 %.....................20 %.....................30 %.....................40 %.....................50 %.....................60 %.....................70 %.....................80 %.....................90 %.....................

In [7]:
def save_spectra(directory_input, firstfile, lastfile, snap_limit, directory_output, Hubble_h):
    
    Age = np.asarray([0.0124, 0.0246, 0.0491, 0.1037, 0.1871, 0.2120, 0.2399, 0.2709, 0.3054, 0.3438, 0.3864, 0.4335, 0.4856, 0.5430, 0.6062, 0.6756, 0.7517, 0.8349, 0.9259, 1.0249, 1.1327, 1.2496, 1.3763, 1.5131, 1.6606, 1.8192, 1.9895, 2.1717, 2.3662, 2.5734, 2.7934, 3.0265, 3.2726, 3.5318, 3.8038, 4.0886, 4.3856, 4.6944, 5.0144, 5.3488, 5.6849, 6.0337, 6.3901, 6.7531, 7.1215, 7.4940, 7.8694, 8.2464, 8.6238, 9.0004, 9.3750, 9.7463, 10.1133, 10.4750, 10.8303, 11.1783, 11.5181, 11.8490, 12.1702, 12.4811, 12.7810, 13.0695, 13.3459, 13.6098])

    for i in range(firstfile, lastfile+1):
        print('Running file number ', i)
        filename = directory_output + "mentari_output_v3_" + str(i) + ".hdf5"
        if os.path.isfile(filename) == 0:

            mass_dusty, metals_dusty = mtr.build_mass_and_metallicity_history(1, directory_input, i, i, snap_limit)
            dust, gas_metals, gas, rad  = mtr.build_dust_history(1, directory_input, i, i, snap_limit)

            #Compute attenuation parameters
            w = np.where((mass_dusty[:,snap_limit] > 0) & (dust[:,snap_limit] > 0))[0]
            Mass = mass_dusty[w] / Hubble_h 
            Metals = metals_dusty[w]

            Dust = dust[w,snap_limit] / Hubble_h
            Gas = gas[w,snap_limit] / Hubble_h
            Rad = rad[w,snap_limit] / Hubble_h

            prescription = 0 #0 for Lagos+ 19; 1 for Somerville+ 12
            tau_BC, eta_BC, tau_ISM, eta_ISM = mtr.compute_attenuation_parameters (prescription, Dust, Gas, Rad)

            #Model Variants 1: Lagos + Dale + Safarzadeh
            time_BC = 10**7
            SSP = 0 #0 for BC03 
            wavelength, spectra, spectra_dusty = mtr.generate_SED(0, Age, Mass, Metals, 
                         tau_BC, tau_ISM, eta_BC, eta_ISM, time_BC)

            wavelength_m1, spectra_m1 = mtr.combine_Dale_SUNRISE(Dust, wavelength, spectra, spectra_dusty)

            #Model Variants 2: Lagos + Dale
            wavelength_m2, spectra_m2 = mtr.add_IR_Dale(wavelength, spectra, spectra_dusty)

            #Model Variants 3: Somerville + Dale + Safarzadeh
            prescription = 1 #0 for Lagos+ 19; 1 for Somerville+ 12
            tau_BC_s, eta_BC_s, tau_ISM_s, eta_ISM_s = mtr.compute_attenuation_parameters (prescription, Dust, Gas, Rad)
            wavelength_s, spectra_s, spectra_dusty_s = mtr.generate_SED(0, Age, Mass, Metals, tau_BC_s, tau_ISM_s, eta_BC_s, eta_ISM_s, time_BC)

            wavelength_m3, spectra_m3 = mtr.combine_Dale_SUNRISE(Dust, wavelength_s, spectra_s, spectra_dusty_s)

            #Model Variants 4: CF00 + Dale + Safarzadeh
            tau_BC_cf = 1.0
            eta_BC_cf = -0.7
            tau_ISM_cf = 0.3
            eta_ISM_cf = -0.7
            wavelength_cf, spectra_cf, spectra_dusty_cf = mtr.generate_SED(0, Age, Mass, Metals, 
                         tau_BC_cf, tau_ISM_cf, eta_BC_cf, eta_ISM_cf, time_BC)

            wavelength_m4, spectra_m4 = mtr.combine_Dale_SUNRISE(Dust, wavelength_cf, spectra_cf, spectra_dusty_cf)

            with h5py.File(filename, 'w') as f:
                f.create_dataset('StellarMass', data=Mass)
                f.create_dataset('Metallicity', data=Metals)
                f.create_dataset('DustMass', data=Dust)
                f.create_dataset('GasMass', data=Gas)
                f.create_dataset('Radius', data=Rad)
                f.create_dataset('Wavelength_m1', data=wavelength_m1)
                f.create_dataset('Spectra_m1', data=spectra_m1)
                f.create_dataset('Wavelength_m2', data=wavelength_m2)
                f.create_dataset('Spectra_m2', data=spectra_m2)
                f.create_dataset('Wavelength_m3', data=wavelength_m3)
                f.create_dataset('Spectra_m3', data=spectra_m3)
                f.create_dataset('Wavelength_m4', data=wavelength_m4)
                f.create_dataset('Spectra_m4', data=spectra_m4)
                f.create_dataset('Wavelength_stellar', data=wavelength)
                f.create_dataset('Spectra_stellar', data=spectra)


In [ ]:
with h5py.File('mentari_test7.h5', 'a') as f:
    f['StellarMass'].resize((f['StellarMass'].shape[0] + Mass.shape[0]), axis=0)
    f['StellarMass'][-Mass.shape[0]:] = Mass
    f['Metallicity'].resize((f['Metallicity'].shape[0] + Metals.shape[0]), axis=0)
    f['Metallicity'][-Metals.shape[0]:] = Metals
    f['DustMass'].resize((f['DustMass'].shape[0] + Dust.shape[0]), axis=0)
    f['DustMass'][-Dust.shape[0]:] = Dust
    f['GasMass'].resize((f['GasMass'].shape[0] + Gas.shape[0]), axis=0)
    f['GasMass'][-Gas.shape[0]:] = Gas
    f['Radius'].resize((f['Radius'].shape[0] + Rad.shape[0]), axis=0)
    f['Radius'][-Rad.shape[0]:] = Rad
    f['Spectra_UVIR'].resize((f['Spectra_UVIR'].shape[0] + spectra_IR.shape[0]), axis=0)
    f['Spectra_UVIR'][-spectra_IR.shape[0]:] = spectra_IR
    f['Spectra_stellar'].resize((f['Spectra_stellar'].shape[0] + spectra.shape[0]), axis=0)
    f['Spectra_stellar'][-spectra.shape[0]:] = spectra
    f['Spectra_SUNRISE'].resize((f['Spectra_SUNRISE'].shape[0] + spectra_IR_b.shape[0]), axis=0)
    f['Spectra_SUNRISE'][-spectra_IR_b.shape[0]:] =spectra_IR_b

## Read the output per file

In [ ]:
mass_dusty = []
metals_dusty = []
wavelength_m1 = []
spectra_m1 = []
wavelength_m2 = []
spectra_m2 = []
wavelength_m3 = []
spectra_m3 = []
wavelength_m4 = []
spectra_m4 = []

dirname = 'output/'
filename = 'mentari_output_'
ext = '.hdf5'
firstfile = 0
lastfile = 7

for i in range(firstfile, lastfile+1):
    file = dirname + filename + str(i) + ext
    print (file)
    with h5py.File(file, 'r') as f:
        mass_dusty.extend(f['StellarMass'])
        metals_dusty.extend(f['Metallicity'])
        wavelength_m1.extend(f['Wavelength_m1'])
        spectra_m1.extend(f['Spectra_m1'])
        wavelength_m2.extend(f['Wavelength_m2'])
        spectra_m2.extend(f['Spectra_m2'])
        wavelength_m3.extend(f['Wavelength_m3'])
        spectra_m3.extend(f['Spectra_m3'])
        wavelength_m4.extend(f['Wavelength_m4'])
        spectra_m4.extend(f['Spectra_m4'])
        
mass_dusty = np.array(mass_dusty)
metals_dusty = np.array(metals_dusty)
spectra_m1 = np.array(spectra_m1)
wavelength_m1 = np.array(wavelength_m1)
spectra_m2 = np.array(spectra_m2)
wavelength_m2 = np.array(wavelength_m2)
spectra_m3 = np.array(spectra_m3)
wavelength_m3 = np.array(wavelength_m3)
spectra_m4 = np.array(spectra_m4)
wavelength_m4 = np.array(wavelength_m4)

                      

In [ ]:
len(wavelength_m2)/8

In [ ]:
plt.figure()
ax = plt.subplot(111)

binwidth = 0.3
Hubble_h = 0.73

w2 = np.where((metals_dusty[:,63] > 0) & (mass_dusty[:,63] / Hubble_h > 0))[0]
mass2 = np.log10(mass_dusty[:,63][w2] )

#c_mass = np.log10(MassHist_old[0]*MetalHist_old[0] * (1. - rec_frac) / Hubble_h) #final computed mass #final computed mass

mi = np.floor(min(mass2)) - 2
ma = np.floor(max(mass2)) + 2
ma = 14
NB = int((ma - mi) / binwidth)

(counts2, binedges2) = np.histogram(mass2, range=(mi, ma), bins=NB)

# Set the x-axis values to be the centre of the bins
xaxeshisto2 = binedges2[:-1] + 0.5 * binwidth

plt.plot(xaxeshisto2, counts2/ (BoxSize/Hubble_h)**3 / binwidth, 'b:', label='dustysage')

# Baldry+ 2008 modified data used for the MCMC fitting
Baldry = np.array([
            [7.05, 1.3531e-01, 6.0741e-02],
            [7.15, 1.3474e-01, 6.0109e-02],
            [7.25, 2.0971e-01, 7.7965e-02],
            [7.35, 1.7161e-01, 3.1841e-02],
            [7.45, 2.1648e-01, 5.7832e-02],
            [7.55, 2.1645e-01, 3.9988e-02],
            [7.65, 2.0837e-01, 4.8713e-02],
            [7.75, 2.0402e-01, 7.0061e-02],
            [7.85, 1.5536e-01, 3.9182e-02],
            [7.95, 1.5232e-01, 2.6824e-02],
            [8.05, 1.5067e-01, 4.8824e-02],
            [8.15, 1.3032e-01, 2.1892e-02],
            [8.25, 1.2545e-01, 3.5526e-02],
            [8.35, 9.8472e-02, 2.7181e-02],
            [8.45, 8.7194e-02, 2.8345e-02],
            [8.55, 7.0758e-02, 2.0808e-02],
            [8.65, 5.8190e-02, 1.3359e-02],
            [8.75, 5.6057e-02, 1.3512e-02],
            [8.85, 5.1380e-02, 1.2815e-02],
            [8.95, 4.4206e-02, 9.6866e-03],
            [9.05, 4.1149e-02, 1.0169e-02],
            [9.15, 3.4959e-02, 6.7898e-03],
            [9.25, 3.3111e-02, 8.3704e-03],
            [9.35, 3.0138e-02, 4.7741e-03],
            [9.45, 2.6692e-02, 5.5029e-03],
            [9.55, 2.4656e-02, 4.4359e-03],
            [9.65, 2.2885e-02, 3.7915e-03],
            [9.75, 2.1849e-02, 3.9812e-03],
            [9.85, 2.0383e-02, 3.2930e-03],
            [9.95, 1.9929e-02, 2.9370e-03],
            [10.05, 1.8865e-02, 2.4624e-03],
            [10.15, 1.8136e-02, 2.5208e-03],
            [10.25, 1.7657e-02, 2.4217e-03],
            [10.35, 1.6616e-02, 2.2784e-03],
            [10.45, 1.6114e-02, 2.1783e-03],
            [10.55, 1.4366e-02, 1.8819e-03],
            [10.65, 1.2588e-02, 1.8249e-03],
            [10.75, 1.1372e-02, 1.4436e-03],
            [10.85, 9.1213e-03, 1.5816e-03],
            [10.95, 6.1125e-03, 9.6735e-04],
            [11.05, 4.3923e-03, 9.6254e-04],
            [11.15, 2.5463e-03, 5.0038e-04],
            [11.25, 1.4298e-03, 4.2816e-04],
            [11.35, 6.4867e-04, 1.6439e-04],
            [11.45, 2.8294e-04, 9.9799e-05],
            [11.55, 1.0617e-04, 4.9085e-05],
            [11.65, 3.2702e-05, 2.4546e-05],
            [11.75, 1.2571e-05, 1.2571e-05],
            [11.85, 8.4589e-06, 8.4589e-06],
            [11.95, 7.4764e-06, 7.4764e-06],
            ], dtype=np.float32)

Baldry_xval = np.log10(10 ** Baldry[:, 0]  /Hubble_h/Hubble_h)
Baldry_xval = Baldry_xval - 0.26  # convert back to Chabrier IMF
Baldry_yvalU = (Baldry[:, 1]+Baldry[:, 2]) * Hubble_h*Hubble_h*Hubble_h
Baldry_yvalL = (Baldry[:, 1]-Baldry[:, 2]) * Hubble_h*Hubble_h*Hubble_h

plt.fill_between(Baldry_xval, Baldry_yvalU, Baldry_yvalL, facecolor='purple', alpha=0.25, label='Baldry et al. 2008 (z=0.1)')

plt.yscale('log', nonposy='clip')
plt.axis([8.0, 13.0, 1.0e-6, 1.0e-1])

# Set the x-axis minor ticks
ax.xaxis.set_minor_locator(plt.MultipleLocator(0.1))

plt.ylabel(r'$\phi\ (\mathrm{Mpc}^{-3}\ \mathrm{dex}^{-1})$')  # Set the y...
plt.xlabel(r'$\log_{10} M_{\mathrm{stars}}\ (M_{\odot})$')  # and the x-axis labels

leg = plt.legend(loc=0, numpoints=1, labelspacing=0.1)
leg.draw_frame(False)  # Don't want a box frame
for t in leg.get_texts():  # Reduce the size of the text
    t.set_fontsize('medium')

plt.savefig('plots/SMF_mini-millennium.png')
plt.show()

In [ ]:
filter_list = ('TwoMass_Ks', 'Sdss_u', 'Sdss_g', 'Sdss_r', 'Sdss_i', 'Sdss_z')
z = 0
#mab_m1 = mtr.compute_mab(wavelength_m1[0:len(spectra_m1[0])], spectra_m1, filter_list, z)
mab_m2 = mtr.compute_mab(wavelength_m2[0:len(spectra_m2[0])], spectra_m2, filter_list, z)
mab_m3 = mtr.compute_mab(wavelength_m3[0:len(spectra_m3[0])], spectra_m3, filter_list, z)
mab_m4 = mtr.compute_mab(wavelength_m4[0:len(spectra_m4[0])], spectra_m4, filter_list, z)


In [ ]:
age = np.asarray([0.0124, 0.0246, 0.0491, 0.1037, 0.1871, 0.2120, 0.2399, 0.2709, 0.3054, 0.3438, 0.3864, 0.4335, 0.4856, 0.5430, 0.6062, 0.6756, 0.7517, 0.8349, 0.9259, 1.0249, 1.1327, 1.2496, 1.3763, 1.5131, 1.6606, 1.8192, 1.9895, 2.1717, 2.3662, 2.5734, 2.7934, 3.0265, 3.2726, 3.5318, 3.8038, 4.0886, 4.3856, 4.6944, 5.0144, 5.3488, 5.6849, 6.0337, 6.3901, 6.7531, 7.1215, 7.4940, 7.8694, 8.2464, 8.6238, 9.0004, 9.3750, 9.7463, 10.1133, 10.4750, 10.8303, 11.1783, 11.5181, 11.8490, 12.1702, 12.4811, 12.7810, 13.0695, 13.3459, 13.6098])
lookbacktime = sorted((np.array([13.6098]*len(age)) - age) * 1.e9)

In [ ]:
w = np.where(mass_dusty[:,63] == max(mass_dusty[:,63]))[0]
mass_dusty_new = list(mass_dusty[w] / Hubble_h)
metals_dusty_new = list(metals_dusty[w])

In [ ]:
wavelength, spectra, spectra_dusty  = mtr.generate_SED(0,age, mass_dusty_new, metals_dusty_new)

In [ ]:
#To plot the spectra:

ax = plt.subplot2grid((1,1), (0,0))
#divider = make_axes_locatable(ax)
ax.plot(np.array(wavelength), spectra[0] * np.array(wavelength), lw=0.5, label='no dust')
ax.plot(np.array(wavelength), spectra_dusty[0] * np.array(wavelength), lw=0.5, label='dusty')

#ax.plot(np.array(wavelength_sage), spectra_sage[0] * np.array(wavelength_sage), lw=0.5, label='sage')
#ax.plot(np.array(wavelength_old), spectra_old[0] * np.array(wavelength_old), lw=0.5, label='sage')

plt.legend()

ax.set_xlabel(r'$\lambda\ (\AA) $')
ax.set_ylabel(r'log $\lambda L_{\lambda} (L_{\odot})$')
ax.set_xscale('log')
ax.set_yscale('log')
plt.savefig('plots/SED-massive.png')

In [ ]:
wavelength_all, spectra_all, spectra_dusty_all = mtr.generate_SED(0,age, mass_dusty / Hubble_h, metals_dusty)

In [ ]:
filter_list = ('TwoMass_Ks', 'Sdss_u', 'Sdss_g', 'Sdss_r', 'Sdss_i', 'Sdss_z')
z = 0.

In [ ]:
mab_dusty = mtr.compute_mab(wavelength_all, spectra_dusty_all, filter_list, z)

In [ ]:
mab = mtr.compute_mab(wavelength_all, spectra_all, filter_list, z)

In [ ]:
hubble=0.73
label = ['Mu', 'Mg', 'Mr', 'Mi', 'Mz']
color = ('grey', 'black', 'blue', 'red', 'green', 'purple')
volume = (BoxSize/Hubble_h)**3

## K-band LF

In [ ]:
mi = -30.0
ma = -15.0
binwidth = 0.3
NB = int((ma - mi) / binwidth)
#M = np.arange(mi, ma, 0.01)

plt.figure()
ax =plt.subplot(111)

# Cole et al. 2001 K band 2dFGRS LF
Cole_Phi = np.array([3.1315561E-03, 8.2625253E-03, 0.0000000E+00, 4.6483092E-03, 5.7576019E-03, 9.1649834E-03, 1.1232893E-02,
            1.0536440E-02, 8.5763102E-03, 8.8181989E-03, 6.9448259E-03, 6.0896124E-03, 9.2596142E-03, 6.9631678E-03,
            7.2867479E-03, 6.9923755E-03, 5.9844730E-03, 5.9305103E-03, 5.3865365E-03, 5.8525647E-03, 5.2373926E-03,
            4.9635037E-03, 4.1801766E-03, 2.7171015E-03, 1.8800517E-03, 1.2136410E-03, 6.5419916E-04, 3.4594961E-04,
            1.4771589E-04, 5.5521199E-05, 2.1283222E-05, 9.4211919E-06, 1.0871951E-06, 2.7923562E-07])
Cole_PhiErr = np.array([3.6377162E-03, 6.6833422E-03, 1.0000000E-10, 4.0996978E-03, 4.3155681E-03, 5.6722397E-03, 6.4211683E-03,
            5.7120644E-03, 4.6346937E-03, 3.8633577E-03, 2.4383855E-03, 1.6279118E-03, 1.6941463E-03, 1.1781409E-03,
            9.7785855E-04, 7.9027453E-04, 6.0649612E-04, 5.1598746E-04, 4.2267537E-04, 3.7395130E-04, 2.8177485E-04,
            2.1805518E-04, 1.6829016E-04, 1.1366483E-04, 8.1871600E-05, 5.7472309E-05, 3.6554517E-05, 2.3141622E-05,
            1.2801432E-05, 6.5092854E-06, 3.3540452E-06, 1.9559407E-06, 5.6035748E-07, 2.8150106E-07])
Cole_Mag = np.array([-18.00000, -18.25000, -18.50000, -18.75000, -19.00000, -19.25000, -19.50000, -19.75000, -20.00000,
            -20.25000, -20.50000, -20.75000, -21.00000, -21.25000, -21.50000, -21.75000, -22.00000, -22.25000,
            -22.50000, -22.75000, -23.00000, -23.25000, -23.50000, -23.75000, -24.00000, -24.25000, -24.50000,
            -24.75000, -25.00000, -25.25000, -25.50000, -25.75000, -26.00000, -26.25000])

# Huang et al. 2003 K band Hawaii+AAO LF
Huang_Phi = np.array([0.0347093, 0.0252148, 0.0437980, 0.0250516, 0.00939655, 0.0193473, 0.0162743, 0.0142267, 0.0174460,
            0.0100971, 0.0136507, 0.00994688, 0.00655286, 0.00528234, 0.00310017, 0.00157789, 0.000721131,
            0.000272634, 8.33409e-05, 2.12150e-05, 3.97432e-06, 5.07697e-06, 5.42939e-07])
Huang_PhiErr = np.array([ 0.0249755, 0.0181685, 0.0161526, 0.0105895, 0.00479689, 0.00525068, 0.00428192, 0.00308970, 0.00248676,
            0.00166458, 0.00166691, 0.00106289, 0.000704721, 0.000527429, 0.000340814, 0.000170548, 8.25681e-05,
            3.81529e-05, 1.50279e-05, 6.16614e-06, 2.34362e-06, 1.98971e-06, 5.54946e-07])
Huang_Mag = np.array([-19.8000, -20.1000, -20.4000, -20.7000, -21.0000, -21.3000, -21.6000, -21.9000, -22.2000, -22.5000,
            -22.8000, -23.1000, -23.4000, -23.7000, -24.0000, -24.3000, -24.6000, -24.9000, -25.2000,
            -25.5000, -25.8000, -26.1000, -26.4000])

# Finally plot the observational data
plt.errorbar(Cole_Mag+5.0*np.log10(Hubble_h), Cole_Phi*Hubble_h*Hubble_h*Hubble_h, yerr=Cole_PhiErr*Hubble_h*Hubble_h*Hubble_h, color='m', lw=1.0, marker='o', ls='none', label='Cole et al. 2001')
plt.errorbar(Huang_Mag+5.0*np.log10(Hubble_h), Huang_Phi*Hubble_h*Hubble_h*Hubble_h, yerr=Huang_PhiErr*Hubble_h*Hubble_h*Hubble_h, color='g', lw=1.0, marker='o', ls='none', label='Huang et al. 2003')

twomass_k = [0]
sdss_ug = [1, 2]
sdss_r = [3]
sdss_i = [4]
sdss_z = [5]
sdss_iz=[4, 5]
sdss =[1,2,3,4,5]

counts, binedges = np.histogram(mab_m1[0]-1.85, range=(mi, ma), bins=NB)
xaxeshisto = binedges[:-1] + 0.5 * binwidth
plt.plot(xaxeshisto, counts/ volume / binwidth, c=color[0], lw=1.0, label='m1')

counts, binedges = np.histogram(mab_m2[0]-1.85, range=(mi, ma), bins=NB)
xaxeshisto = binedges[:-1] + 0.5 * binwidth
plt.plot(xaxeshisto, counts/ volume / binwidth, '--', c=color[0], lw=1.0, label='m2')

counts, binedges = np.histogram(mab_m3[0]-1.85, range=(mi, ma), bins=NB)
xaxeshisto = binedges[:-1] + 0.5 * binwidth
plt.plot(xaxeshisto, counts/ volume / binwidth, ':', c=color[0], lw=1.0, label='m3')

counts, binedges = np.histogram(mab_m4[0]-1.85, range=(mi, ma), bins=NB)
xaxeshisto = binedges[:-1] + 0.5 * binwidth
plt.plot(xaxeshisto, counts/ volume / binwidth, '-.', c=color[0], lw=1.0, label='m4')

plt.yscale('log', nonposy='clip')
plt.ylabel(r'$\phi\ (\mathrm{Mpc}^{-3}\ \mathrm{mag}^{-1})$')  # Set the y...
plt.xlabel(r'$M_\mathrm{K}$')
plt.ylim(4e-7, 4e-2)
plt.xlim(-19.5, -27.5)
ax.xaxis.set_minor_locator(plt.MultipleLocator(0.1))
leg = plt.legend(loc=0, numpoints=1, labelspacing=0.1)
leg.draw_frame(False)  # Don't want a box frame
for t in leg.get_texts():  # Reduce the size of the text
    t.set_fontsize('medium')
plt.savefig('plots/2mass_k_mini.png')
plt.show()

In [ ]:
mi = -30.0
ma = -15.0
binwidth = 0.3
NB = int((ma - mi) / binwidth)
M = np.arange(mi, ma, 0.1)
label = ['Mk', 'Mu', 'Mg', 'Mr', 'Mi', 'Mz']
# Observations from Kelvin et al. 2013
Mstar = [0, -18.53, -20.28, -20.90, -21.45, -21.78]
alpha1 = [0, -0.91, -1.29, -1.13, -1.35, -1.46]
phistar1 = [0, 0.00964, 0.00351, 0.00451, 0.00220, 0.0014]
alpha2 = [0, 1.25, 0.06, 0.53, -0.09, -0.26]
phistar2 = [0, 0.00146, 0.00488, 0.00301, 0.00487, 0.00505]

plt.figure(figsize=(16,8))
row = 2
col = 3
for i in range(1, 6):
    ax =plt.subplot(row, col, i)
    
    counts, binedges = np.histogram(mab_m1[i], range=(mi, ma), bins=NB)
    xaxeshisto = binedges[:-1] + 0.5 * binwidth
    plt.plot(xaxeshisto, counts/ volume / binwidth, c=color[i], lw=1.0, label='m1')
    
    counts, binedges = np.histogram(mab_m2[i], range=(mi, ma), bins=NB)
    xaxeshisto = binedges[:-1] + 0.5 * binwidth
    plt.plot(xaxeshisto, counts/ volume / binwidth, '--', c=color[i], lw=1.0, label='m2')
    
    counts, binedges = np.histogram(mab_m3[i], range=(mi, ma), bins=NB)
    xaxeshisto = binedges[:-1] + 0.5 * binwidth
    plt.plot(xaxeshisto, counts/ volume / binwidth, ':', c=color[i], lw=1.0, label='m3')
    
    counts, binedges = np.histogram(mab_m4[i], range=(mi, ma), bins=NB)
    xaxeshisto = binedges[:-1] + 0.5 * binwidth
    plt.plot(xaxeshisto, counts/ volume / binwidth, '-.', c=color[i],lw=1.0, label='m4')
    
    #plot Kelvin+ 13 dataset
    M_plot = M + 0.65 #convert back to Chabrier IMF
    xval = 10.0 ** (0.4*(Mstar[i]-M)) 
    yval = 0.4 * np.log(10.0) * (phistar1[i] * xval ** (alpha1[i]+1) + phistar2[i] * xval ** (alpha2[i]+1)) * np.exp(-xval)                
    plt.plot(M_plot, yval, '--', c=color[i], lw=0.5, label='Kelvin+ 13')

    plt.yscale('log', nonposy='clip')
    plt.ylim(2e-7, 2e-2)
    plt.xlim(-17.1, -24.9)
    ax.tick_params(axis='both', which='both', direction='in')
    ax.xaxis.set_minor_locator(plt.MultipleLocator(0.1))
    plt.ylabel(r'$\phi\ (\mathrm{Mpc}^{-3}\ \mathrm{dex}^{-1})$')  # Set the y...
    plt.xlabel(label[i])


leg = plt.legend(loc=0, numpoints=1, labelspacing=0.1)
leg.draw_frame(False)  # Don't want a box frame
for t in leg.get_texts():  # Reduce the size of the text
    t.set_fontsize('medium')
plt.savefig('plots/sdss-mini.png')
#plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

In [ ]:
IR_filter = ('IRAC_1','IRAC_4', 'MIPS_24um', 'MIPS_70um','PACS_160um', 'SPIRE_250um', 'SPIRE_500um')
z=0
mab_IR_m1 = mtr.compute_mab(wavelength_m1[0:len(spectra_m1[0])], spectra_m1, IR_filter, z)
mab_IR_m2 = mtr.compute_mab(wavelength_m2[0:len(spectra_m2[0])], spectra_m2, IR_filter, z)
mab_IR_m3 = mtr.compute_mab(wavelength_m3[0:len(spectra_m3[0])], spectra_m3, IR_filter, z)
mab_IR_m4 = mtr.compute_mab(wavelength_m4[0:len(spectra_m4[0])], spectra_m4, IR_filter, z)


In [ ]:
lowlim_list = [-25.3, -25.3, -23.8]
uplim_list = [-29, -29, -28]

row = 4
col = 2
fig, axes = plt.subplots(row, col, sharey=True, figsize=(9,12))

mi = -30.0
ma = -20.0
binwidth = 0.5
NB = int((ma - mi) / binwidth)

for i in range(0, 6):
    ax = plt.subplot(row, col, i+1)
    counts1, binedges1 = np.histogram(mab_IR_m1[i], range=(mi, ma), bins=NB)
    xaxeshisto1 = binedges1[:-1] + 0.5 * binwidth
    plt.plot(xaxeshisto1, counts1/ ((BoxSize/Hubble_h)**3) / binwidth, '-', lw=1.5)

    counts1, binedges1 = np.histogram(mab_IR_m2[i], range=(mi, ma), bins=NB)
    xaxeshisto1 = binedges1[:-1] + 0.5 * binwidth
    plt.plot(xaxeshisto1, counts1/ ((BoxSize/Hubble_h)**3) / binwidth, '--', lw=1.5)
    
    counts1, binedges1 = np.histogram(mab_IR_m3[i], range=(mi, ma), bins=NB)
    xaxeshisto1 = binedges1[:-1] + 0.5 * binwidth
    plt.plot(xaxeshisto1, counts1/ ((BoxSize/Hubble_h)**3) / binwidth, ':', lw=1.5)
    
    counts1, binedges1 = np.histogram(mab_IR_m4[i], range=(mi, ma), bins=NB)
    xaxeshisto1 = binedges1[:-1] + 0.5 * binwidth
    plt.plot(xaxeshisto1, counts1/ ((BoxSize/Hubble_h)**3) / binwidth, '-.', lw=1.5)
    
    plt.annotate(IR_filter[i], xy=(0.7, 0.05), xycoords='axes fraction')
    plt.yscale('log', nonposy='clip')
    plt.ylim(2e-5, 2e-2)
    plt.xlim(-30.5, -20.5)
    ax.tick_params(axis='both', which='both', direction='in')
    ax.xaxis.set_minor_locator(plt.MultipleLocator(0.1))
    plt.ylabel(r'$\phi\ (\mathrm{Mpc}^{-3}\ \mathrm{dex}^{-1})$')  # Set the y...

ax = plt.subplot(row, col, 7)
counts1, binedges1 = np.histogram(mab_IR_m1[6], range=(mi, ma), bins=NB)
xaxeshisto1 = binedges1[:-1] + 0.5 * binwidth
plt.plot(xaxeshisto1, counts1/ ((BoxSize/Hubble_h)**3) / binwidth, '-', lw=1.5, label='m1')

counts1, binedges1 = np.histogram(mab_IR_m2[6], range=(mi, ma), bins=NB)
xaxeshisto1 = binedges1[:-1] + 0.5 * binwidth
plt.plot(xaxeshisto1, counts1/ ((BoxSize/Hubble_h)**3) / binwidth, '--', lw=1.5, label='m2')

counts1, binedges1 = np.histogram(mab_IR_m3[6], range=(mi, ma), bins=NB)
xaxeshisto1 = binedges1[:-1] + 0.5 * binwidth
plt.plot(xaxeshisto1, counts1/ ((BoxSize/Hubble_h)**3) / binwidth, ':', lw=1.5, label='m3')

counts1, binedges1 = np.histogram(mab_IR_m4[6], range=(mi, ma), bins=NB)
xaxeshisto1 = binedges1[:-1] + 0.5 * binwidth
plt.plot(xaxeshisto1, counts1/ ((BoxSize/Hubble_h)**3) / binwidth, '-.', lw=1.5, label='m4')

plt.annotate(IR_filter[6], xy=(0.7, 0.05), xycoords='axes fraction')
plt.yscale('log', nonposy='clip')
plt.ylim(2e-5, 2e-2)
plt.xlim(-30.5, -20.5)
ax.tick_params(axis='both', which='both', direction='in')
ax.xaxis.set_minor_locator(plt.MultipleLocator(0.1))
plt.ylabel(r'$\phi\ (\mathrm{Mpc}^{-3}\ \mathrm{dex}^{-1})$')  # Set the y...


plt.subplot(421)
Dai_mag = [-21.852, -21.591, -21.348, -21.087, -20.850, -20.619, -20.351, -20.157,
          -19.886, -19.596, -19.347, -19.090, -18.855, -18.602, -18.364, -18.114,
          -17.854, -17.598, -17.351, -17.127, -16.833, -16.600, -16.344, -16.101, 
          -15.862, -15.566]
Dai_phi = 10 ** np.array([-3.527, -3.028, -2.935, -2.674, -2.548, -2.389, -2.300, -2.197, -2.148, -2.046,
          -2.032, -1.974, -1.944, -1.957, -1.718, -1.832, -1.766, -1.825, -1.830, -1.469,
          -1.198, -1.408, -1.574, -1.701, -1.020, -1.027])
plt.errorbar(Dai_mag+5.0*np.log10(Hubble_h), Dai_phi*Hubble_h*Hubble_h*Hubble_h, lw=1.0, marker='o', ls='none',label='Dai et al. 2009')
plt.legend(loc='upper left')
plt.xlim(-26.5, -16.5)

plt.subplot(422).set_yticklabels([])
plt.subplot(422).set_ylabel('')
Dai_mag = [-23.768, -23.495, -23.266, -22.995, -22.734, -22.466, -22.265, -21.976,
          -21.790, -21.519, -21.264, -20.966, -20.752, -20.479, -20.274, -19.986,
          -19.718, -19.493, -19.220, -18.996, -18.749, -18.491, -18.247, -18.002,
          -17.745, -17.491, -17.265, -16.762, -16.013, -16.985]
Dai_phi = 10 ** np.array([-4.235, -3.838, -3.500, -3.339, -3.038, -2.803, -2.712,
                         -2.598, -2.523, -2.448, -2.388, -2.300, -2.253, -2.237,
                          -2.084, -2.092, -2.068, -1.921, -2.007, -1.799, -1.784,
                         -1.767, -1.686, -1.422, -1.371, -1.596, -1.120, -1.203,
                         -1.566, -1.863])
plt.errorbar(Dai_mag+5.0*np.log10(Hubble_h), Dai_phi*Hubble_h*Hubble_h*Hubble_h, lw=1.0, marker='o', ls='none',label='Dai et al. 2009')
plt.legend(loc='upper left')
plt.xlim(-26.5, -16.5)

plt.subplot(423)
plt.legend(loc='upper left')
plt.xlim(-26.5, -20.5)

plt.subplot(424).set_yticklabels([])
plt.subplot(424).set_ylabel('')
plt.legend(loc='upper left')

plt.subplot(425)
#Marchetti_mag, Marchetti_phi = np.loadtxt('files/Marchetti_PACS160.dat', usecols=(0,1), unpack=True)
Marchetti_mag = [-27.811, -27.373, -26.953, -26.573, -26.031, -25.686, -25.271, -24.844,
                -24.413, -23.998, -23.596, -23.121, -22.819, -22.347]
Marchetti_phi = 10 ** np.array([-4.123, -3.558, -3.032, -2.698, -2.451, -2.320, -2.268, -2.071, -2.047, -2.058,
                 -2.031, -2.099, -1.858, -1.946])

plt.errorbar(Marchetti_mag+5.0*np.log10(Hubble_h), Marchetti_phi*Hubble_h*Hubble_h*Hubble_h, lw=1.0, marker='o', ls='none',label='Marchetti et al. 2016')

Patel_mag = [-27.693, -26.674, -25.695, -24.684, -23.721, -22.664]
Patel_phi = 10 ** np.array([-3.789, -3.018, -2.590, -2.501, -2.802, -1.660])
plt.errorbar(Patel_mag+5.0*np.log10(Hubble_h), Patel_phi*Hubble_h*Hubble_h*Hubble_h, lw=1.0, marker='o', ls='none',label='Patel et al. 2013')
plt.legend(loc='upper left')


plt.subplot(426).set_yticklabels([])
plt.subplot(426).set_ylabel('')
Marchetti_mag = [-26.597, -26.222, -25.731, -25.342, -24.946, -24.505,
                -24.066, -23.698, -23.257, -22.841, -22.452, -22.046, -21.620]
Marchetti_phi = 10 ** np.array([-3.584, -3.074, -2.665, -2.246, -2.020,
                               -1.786, -1.671, -1.713, -1.707, -1.601, -1.649,
                               -1.492, -1.636])
plt.errorbar(Marchetti_mag+5.0*np.log10(Hubble_h), Marchetti_phi*Hubble_h*Hubble_h*Hubble_h, lw=1.0, marker='o', ls='none',label='Marchetti et al. 2016')

Dye_mag = [-26.287, -25.574, -24.838, -24.107, -23.421, -22.679, -21.986]
Dye_phi = 10 ** np.array([-3.410, -2.872, -2.591, -2.508, -2.263, -1.966, -1.927])
plt.errorbar(Dye_mag+5.0*np.log10(Hubble_h), Dye_phi*Hubble_h*Hubble_h*Hubble_h, lw=1.0, marker='o', ls='none',label='Dya et al. 2010')
plt.legend(loc='upper left')

plt.subplot(427)
Marchetti_mag = [-24.274, -23.888, -23.522, -23.043, -22.633, -22.208, -21.764,
                 -21.409, -21.019, -20.568, -20.129]
Marchetti_phi = 10 ** np.array([-3.606, -2.985, -2.545, -2.422, -2.149, -2.093,
                               -2.013, -2.096, -2.083, -1.919, -1.996])
plt.errorbar(Marchetti_mag+5.0*np.log10(Hubble_h), Marchetti_phi*Hubble_h*Hubble_h*Hubble_h, lw=1.0, marker='o', ls='none',label='Marchetti et al. 2016')

Negrello_mag = [-24.750, -24.278, -23.551, -22.719, -21.975, -21.238, -20.498, -19.731]
Negrello_phi = 10 ** np.array([-3.688, -3.592, -2.816, -2.267, -1.990, -1.883, -1.656, -1.388])
plt.errorbar(Negrello_mag+5.0*np.log10(Hubble_h), Negrello_phi*Hubble_h*Hubble_h*Hubble_h, lw=1.0, marker='o', ls='none',label='Negrello et al. 2013')

fig.delaxes(axes[-1,-1]) 
plt.legend(loc='upper left')

plt.xlabel('mAB')
plt.subplots_adjust(wspace=0)
plt.savefig('plots/IR_mini.png')


In [ ]:
mi = -30.0
ma = -15.0
binwidth = 0.4
NB = int((ma - mi) / binwidth)
M = np.arange(mi, ma, 0.1)

plt.figure()
ax =plt.subplot(111)

# Observations from Kelvin et al. 2013
'''
# SDSS u
Mstar = -18.53
alpha1 = -0.91
phistar1 = 0.00964
alpha2 = 1.25
phistar2 = 0.00146
xval = 10.0 ** (0.4*(Mstar-M))
yval = 0.4 * np.log(10.0) * (phistar1 * xval ** (alpha1+1) + phistar2 * xval ** (alpha2+1)) * np.exp(-xval)      
plt.plot(M, yval, '--', c=color[1], lw=0.5, label='Kelvin et al (2013)')

# SDSS g
Mstar = -20.28
alpha1 = -1.29
phistar1 = 0.00351
alpha2 = 0.06
phistar2 = 0.00488
xval = 10.0 ** (0.4*(Mstar-M))
yval = 0.4 * np.log(10.0) * (phistar1 * xval ** (alpha1+1) + phistar2 * xval ** (alpha2+1)) * np.exp(-xval)          
plt.plot(M, yval, '--', c=color[2],lw=0.5)
'''
# SDSS r
Mstar = -20.90
alpha1 = -1.13
phistar1 = 0.00451
alpha2 = 0.53
phistar2 = 0.00301
xval = 10.0 ** (0.4*(Mstar-M))
yval = 0.4 * np.log(10.0) * (phistar1 * xval ** (alpha1+1) + phistar2 * xval ** (alpha2+1)) * np.exp(-xval)          
plt.plot(M, yval, '--', c=color[3],lw=0.5, label='Kelvin et al (2013)')
'''
# SDSS i
Mstar = -21.45
alpha1 = -1.35
phistar1 = 0.00220
alpha2 = -0.09
phistar2 = 0.00487
xval = 10.0 ** (0.4*(Mstar-M))
yval = 0.4 * np.log(10.0) * (phistar1 * xval ** (alpha1+1) + phistar2 * xval ** (alpha2+1)) * np.exp(-xval)              
plt.plot(M, yval, '--', c=color[4], lw=0.5, label='Kelvin et al (2013)')

# SDSS z
Mstar = -21.78
alpha1 = -1.46
phistar1 = 0.0014
alpha2 = -0.26
phistar2 = 0.00505
xval = 10.0 ** (0.4*(Mstar-M))
yval = 0.4 * np.log(10.0) * (phistar1 * xval ** (alpha1+1) + phistar2 * xval ** (alpha2+1)) * np.exp(-xval)                
plt.plot(M, yval, '--', c=color[5], lw=0.5)
'''
#Dataset from Blanton et al. (2003) (LCDM)
k_correct_zeq0 = -2.5*np.log(1.1)  # shift observations from z=0.1 to z=0.0 (Blanton et al. 2003 eq.4)
'''
# SDSS u
Mstar = -17.93 +5.0*np.log10(Hubble_h) +k_correct_zeq0
alpha = -0.92
phistar = 0.0305 *Hubble_h*Hubble_h*Hubble_h
xval = 10.0 ** (0.4*(Mstar-M))
yval = 0.4 * np.log(10.0) * phistar * xval ** (alpha+1) * np.exp(-xval)      
plt.plot(M, yval, '--', c=color[1], lw=1.0, label='Blanton et al (2003)')

# SDSS g
Mstar = -19.39 +5.0*np.log10(Hubble_h) +k_correct_zeq0
alpha = -0.89
phistar = 0.0218 *Hubble_h*Hubble_h*Hubble_h
xval = 10.0 ** (0.4*(Mstar-M))
yval = 0.4 * np.log(10.0) * phistar * xval ** (alpha+1) * np.exp(-xval)      
plt.plot(M, yval,'--', c=color[2], lw=1.0)
'''
# SDSS r
Mstar = -20.44 +5.0*np.log10(Hubble_h) +k_correct_zeq0
alpha = -1.05
phistar = 0.0149 *Hubble_h*Hubble_h*Hubble_h
xval = 10.0 ** (0.4*(Mstar-M))
yval = 0.4 * np.log(10.0) * phistar * xval ** (alpha+1) * np.exp(-xval)      
plt.plot(M, yval,'--', c=color[3], lw=1.0)
'''
# SDSS i
Mstar = -20.82 +5.0*np.log10(Hubble_h) +k_correct_zeq0
alpha = -1.00
phistar = 0.0147 *Hubble_h*Hubble_h*Hubble_h
xval = 10.0 ** (0.4*(Mstar-M))
yval = 0.4 * np.log(10.0) * phistar * xval ** (alpha+1) * np.exp(-xval)      
plt.plot(M, yval, '--', c=color[4], lw=1.0, label='Blanton et al (2003)')

# SDSS z
Mstar = -21.18 +5.0*np.log10(Hubble_h) +k_correct_zeq0
alpha = -1.08
phistar = 0.0135 *Hubble_h*Hubble_h*Hubble_h
xval = 10.0 ** (0.4*(Mstar-M))
yval = 0.4 * np.log(10.0) * phistar * xval ** (alpha+1) * np.exp(-xval)      
plt.plot(M, yval, '--', c=color[5], lw=1.0)
'''
twomass_k = [0]
sdss_ug = [1, 2]
sdss_r = [3]
sdss_i = [4]
sdss_z = [5]
sdss_iz=[4, 5]
sdss =[1,2,3,4,5]

for i in sdss_r:
    
    counts, binedges = np.histogram(mab_dusty[i], range=(mi, ma), bins=NB)
    xaxeshisto = binedges[:-1] + 0.5 * binwidth
    plt.plot(xaxeshisto, counts/ ((BoxSize/Hubble_h)**3) / binwidth, c=color[i], lw=1.5)

    counts, binedges = np.histogram(mab[i], range=(mi, ma), bins=NB)
    xaxeshisto = binedges[:-1] + 0.5 * binwidth
    plt.plot(xaxeshisto, counts/ ((BoxSize/Hubble_h)**3) / binwidth, '--', c=color[i], lw=1.5)
'''
counts, binedges = np.histogram(mab_dusty[1], range=(mi, ma), bins=NB)
xaxeshisto = binedges[:-1] + 0.5 * binwidth
plt.plot(xaxeshisto, counts/ ((BoxSize/Hubble_h)**3) / binwidth, c=color[1], lw=1.5, label='dusty')

counts, binedges = np.histogram(mab[1], range=(mi, ma), bins=NB)
xaxeshisto = binedges[:-1] + 0.5 * binwidth
plt.plot(xaxeshisto, counts/ ((BoxSize/Hubble_h)**3) / binwidth, '--', c=color[1], lw=1.5, label='no-dust')
'''
plt.yscale('log', nonposy='clip')
plt.ylabel(r'$\phi\ (\mathrm{Mpc}^{-3}\ \mathrm{dex}^{-1})$')  # Set the y...
plt.xlabel(r'$M$')
plt.ylim(2e-7, 2e-2)
plt.xlim(-17.0, -25.0)
ax.xaxis.set_minor_locator(plt.MultipleLocator(0.1))
leg = plt.legend(loc=0, numpoints=1, labelspacing=0.1)
leg.draw_frame(False)  # Don't want a box frame
for t in leg.get_texts():  # Reduce the size of the text
    t.set_fontsize('medium')
#plt.savefig('plots/sdss-iz-mini.png')
plt.show()

In [ ]:
w = np.where((mass_dusty[:,snap_limit] > 0) & (dust[:,snap_limit] == 0))[0]

Mass = mass_dusty[w] / Hubble_h 
Metals = metals_dusty[w]
Dust = dust[w,snap_limit] / Hubble_h
Gas = gas[w,snap_limit] / Hubble_h
Rad = rad[w,snap_limit] / Hubble_h
eta_BC = [-0.7] * len(Dust)
eta_ISM_v2 = [-1.3] * len(Dust)
tau_BC = [0.0] * len(Dust)
tau_ISM = [0.0] * len(Dust)
#prescription = 0 #0 for Lagos+ 19; 1 for Somerville+ 12
#tau_BC, eta_BC, tau_ISM, eta_ISM = mtr.compute_attenuation_parameters (prescription, Dust, Gas, Rad)
wavelength, spectra, spectra_dusty = mtr.generate_SED(0, Age, Mass, Metals, 
             tau_BC, tau_ISM, eta_BC, eta_ISM_v2, time_BC)
